In [18]:
import numpy as np
import pandas as pd
import pickle

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import KFold

pd.set_option('display.max_columns', None)

In [2]:
pickle_in = open("processed_film_data","rb")
film_df = pickle.load(pickle_in)

In [3]:
film_df.head()

,actors,budget,countries,directors,genres,gross_usa,languages,mpaa,rating,runtime,votes,writers
0,"[Tom Selleck, Laura San Giacomo, Alan Rickman]",20000000,+_+Australia+_+USA,Simon Wincer,"[Action, Adventure, Drama, Romance, Western]",21413105,"[English, Aboriginal]",PG-13,6.8,119.0,16501,John Hill
1,"[Kevin Costner, Mary McDonnell, Graham Greene]",22000000,+_+USA+_+UK,Kevin Costner,"[Adventure, Drama, Western]",184208848,"[English, Sioux, Pawnee]",PG-13,8.0,181.0,204981,Michael Blake
2,"[Tony Todd, Patricia Tallman, Tom Towles]",4200000,+_+USA,Tom Savini,[Horror],5835247,[English],R,6.9,92.0,33738,John A. Russo
3,"[Jonathan Brandis, Kenny Morrison, Clarissa Burt]",36000000,+_+USA+_+Germany,George Miller,"[Adventure, Drama, Family, Fantasy]",17373527,[English],PG,5.1,90.0,20152,Karin Howard
4,"[David Andrews, Kelly Wolf, Stephen Macht]",10500000,+_+USA+_+Japan,Ralph S. Singleton,[Horror],11582891,[English],R,4.8,89.0,7824,Stephen King


In [4]:
#Create dummy columns for genres
genres = film_df['genres'].str.join('|').str.get_dummies()
genres.reset_index(drop=True, inplace=True)
film_df = pd.concat([film_df, genres], axis=1)
del film_df['genres']

In [5]:
#Create dummy columns for mpaa
mpaa = film_df['mpaa'].str.get_dummies()
mpaa.reset_index(inplace=True, drop=True)
film_df = pd.concat([film_df, mpaa], axis=1)
del film_df['mpaa']

In [6]:
#Create dummy columns for languages
languages = film_df['languages'].str.join('|').str.get_dummies()

In [7]:
#Check the values
languages.sum().sort_values(ascending = False).head(10)

English     4453
Spanish      446
French       328
German       195
Italian      165
Russian      157
Japanese     102
Arabic        87
Mandarin      78
Latin         53
dtype: int64

In [8]:
#Pick the top 7 languages and add them back to the dataframe
languages = languages[['English', 'Spanish', 'French', 'German', 'Italian', 'Russian', 'Japanese']]
film_df = pd.concat([film_df, languages], axis=1)
del film_df['languages']

In [9]:
#Look at director percentiles
film_df['directors'].value_counts().quantile(0.9)

5.0

In [10]:
#Create dummy columns for directors and delete all who have fewer than 5 films.
directors = film_df['directors'].str.get_dummies()
directors = directors[directors.columns[directors.sum() >=5]]
film_df = pd.concat([film_df, directors], axis=1)
del film_df['directors']

In [11]:
#Look at writers now
film_df['writers'].value_counts().head()

Woody Allen          22
Stephen King         15
Joel Coen            11
Michael Crichton     10
Quentin Tarantino    10
Name: writers, dtype: int64

In [12]:
#Check writer percentiles
film_df['writers'].value_counts().quantile(0.9)

3.0

In [13]:
#Remove writers who have fewer than 5 movies.
writers = film_df['writers'].str.get_dummies()
writers = writers[writers.columns[writers.sum() >=5]]
film_df = pd.concat([film_df, writers], axis=1)
del film_df['writers']

In [14]:
#Create dummy columns for actors
actors = film_df['actors'].str.join('|').str.get_dummies()

In [15]:
#Check which actors appear in movies the most.
actors.sum().quantile(0.9)

6.0

In [16]:
#Select these actors.
actors = actors[actors.columns[actors.sum() >=6]]
film_df = pd.concat([film_df, actors], axis=1)
del film_df['actors']

In [17]:
#Create dummy columns for countries
countries = film_df['countries'].str.join('|').str.get_dummies()
countries.sum().sort_values(ascending = False).head(10)
countries = countries[['USA', 'UK', 'Germany', 'Canada', 'France']]
film_df = pd.concat([film_df, countries], axis=1)
del film_df['countries']

KeyError: "['USA' 'UK' 'Germany' 'Canada' 'France'] not in index"

In [ ]:
#Apply log based on previous observations (based on observations in Modeling 1)
film_df['budget'] = film_df['budget'].apply(lambda x: np.log(x))
film_df['gross_usa'] = film_df['gross_usa'].apply(lambda x: np.log(x))
film_df['runtime'] = film_df['runtime'].apply(lambda x: np.log(x))
film_df['votes'] = film_df['votes'].apply(lambda x: np.log(x))

In [ ]:
#Scale data
scaler = MinMaxScaler()

data = film_df.drop('rating', axis=1)
target = film_df['rating']

data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [ ]:
#Look at the correlation matrix
film_df.corr()

# Diagnostics

In [ ]:
#Diagnostic plots
plt.figure(figsize=(15,5))

X, y = data, target

rgr = LinearRegression()
rgr.fit(X, y)
pred = rgr.predict(X)

plt.subplot(1, 3, 1)
plt.scatter(pred, y, alpha = 0.1)
plt.title("Regular regression fit")
plt.xlabel("pred")
plt.ylabel("y")

plt.subplot(1, 3, 2)
res = (y - pred)
plt.scatter(pred, res, alpha = 0.1)
plt.title("Regular regression residual plot")
plt.xlabel("prediction")
plt.ylabel("residuals")

plt.subplot(1, 3, 3)
pred = rgr.predict(X)
stats.probplot(res, dist="norm", plot=plt)
plt.title("Regular regression Q-Q plot");

In [ ]:
#Statmodels analysis
model = sm.OLS(y,X)
fit = model.fit()
fit.summary()

# Models

In [ ]:
#This function will help me calculate the RMSE
def RMSE(actuals, preds): 
    return np.sqrt(np.mean((actuals - preds)**2))

In [ ]:
#Divide the data into training, test, and validation sets.
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=42) 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

kf = KFold(n_splits=5, shuffle=True, random_state = 71)

## Model 1 - Basic Linear Regression

In [ ]:
#Initialize the model
lr_model_1 = LinearRegression()
lr_model_1.fit(X_train, y_train)

In [ ]:
#Perform basic validation and look at the results.
plt.scatter(lr_model_1.predict(X_val), y_val, alpha=.1)
plt.plot(np.linspace(0,12,0.1),np.linspace(0,12,0.1))
plt.xlabel("validation predictions")
plt.ylabel("validation true values");

In [ ]:
#Look at the model's RMSE and r^2
print("Basic Linear Regression RMSE: %.3f" %(RMSE(y_val, lr_model_1.predict(X_val))))
print("Basic Linear Regression r^2: %.3f" %(lr_model_1.score(X_val, y_val)))

In [ ]:
#Perform cross-validation
X_cv, y_cv = np.array(X), np.array(y)

cv_lm_r2s = []
cv_lm_rmse = []

for train_ind, val_ind in kf.split(X_cv, y_cv):
    
    X_cv_train, y_cv_train = X_cv[train_ind], y_cv[train_ind]
    X_val, y_val = X_cv[val_ind], y_cv[val_ind] 
    
    lr_model_1.fit(X_cv_train, y_cv_train)
    
    cv_lm_r2s.append(lr_model_1.score(X_val, y_val))
    cv_lm_rmse.append(RMSE(y_val, lr_model_1.predict(X_val)))

#Check RMSE and r^2 for cross-validation.
print('Basic Linear Regression mean cv r^2: %.3f +- %.3f' %(np.mean(cv_lm_r2s),np.std(cv_lm_r2s)))
print('Basic Linear Regression mean cv RMSE: %.3f +- %.3f' %(np.mean(cv_lm_rmse),np.std(cv_lm_rmse)))

In [ ]:
#Check r^2 on the test set.
lr_model_1.score(X_test, y_test)

## Model 2 - LASSO regularization

In [ ]:
#Define alphavec for LASSO and Ridge CV
alphavec = 10**np.linspace(-5,5,10)

In [ ]:
#Scale the training set
std = StandardScaler()
std.fit(X_train);

In [ ]:
#Transform the training data for LASSO regularization
X_lasso_train = std.transform(X_train)
X_lasso_val = std.transform(X_val)

In [ ]:
#Initialize the model and perform cross-validation using in-built methods.
lr_model_2 = LassoCV(alphas = alphavec, cv=5)
lr_model_2.fit(X_lasso_train,y_train)

In [ ]:
#Look at the coefficients.
list(zip(X_train.columns, lr_model_2.coef_))

In [ ]:
#Look at a single validation sert.
val_set_pred_2 = lr_model_2.predict(X_lasso_val)

In [ ]:
#Check RMSE and r^2 for LASSO regularization for one validation set.
print('LASSO mean cv r^2: %.3f' %lr_model_2.score(X_lasso_val, y_val))
print('LASSO mean cv RMSE: %.3f' %np.sqrt(np.mean((val_set_pred_2 - y_val)**2)))

In [ ]:
#Check the model's r^2 on the test data.
lr_model_2.score(std.transform(X_test), y_test)

## Model 3 - Ridge Regularization

In [ ]:
#Scale the data
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
#Initialize the model and perform built-in cross-validation.
lr_model_3 = RidgeCV(alphas = alphavec, cv=5)
lr_model_3.fit(X_train_scaled,y_train)

In [ ]:
#Check the coefficients
list(zip(X_train.columns,lr_model_3.coef_))

In [ ]:
#Look at one validation set...
val_set_pred_3 = lr_model_3.predict(X_val_scaled)

In [ ]:
#... and check its RMSE and r^2
print('Ridge mean cv r^2: %.3f' %lr_model_3.score(X_val_scaled, y_val))
print('Ridge mean cv RMSE: %.3f' %np.sqrt(np.mean((val_set_pred_3 - y_val)**2)))

In [ ]:
#Test the model on the test set
lr_model_3.score(std.transform(X_test), y_test)

## Model 4 - Linear Regression With Polynomials

I will not perform polynomial regression on this set because it is computationally intensive, and based on my work in notebook 3, it will not yield any meaningful results.